In [ ]:
#https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python''

이 노트북은 기초 학습 모델, 특히 스택으로 알려진 앙상블의 변형에 대한 매우 기본적이고 단순한 입문 프라이머다. 간단히 말하면, 1단계(베이스)로서 적층 용도를 사용하는 경우, 몇 가지 기본 분류기의 예측을 한 다음, 
2단계에서 다른 모델을 사용하여 이전의 1단계 예측으로부터 산출물을 예측한다.

타이타닉 데이터 세트는 카글의 많은 신인들이 이곳에서 출발하기 때문에 이 개념을 도입하는 데 있어 가장 유력한 후보지 이다. 게다가, 비록 많은 팀이 카글 대회에서 우승한 책임이 있지만, 이 주제에 대한 알맹이가 부족한 것 같아, 이 공책이 그 공백을 어느 정도 메울 수 있기를 바란다.

나 자신도 캐글에 새로 들어온 사람이고, 내가 우연히 우연히 알게 된 첫 번째 적절한 앙상블/추적 대본은 위대한 파론의 '올스테이트 심각성 주장 대회'에 쓰여진 것이었다. 이 노트북의 자료는 파론의 대본에서 많이 빌려온다. 그가 퇴행자의 앙상블이었을 때 분류자의 앙상블을 고려하기 위해 배치되었다. 어쨌든, 그의 대본을 여기서 확인해보자.

 
이제 손에 들고 있는 노트북에, 나는 그것이 정의를 행하고 직관적이고 간결한 방식으로 앙상블의 개념을 전달하기를 바란다. 아래에 설명된 것과 정확히 동일한 앙상블 단계를 구현하는 나의 다른 독립형 캐글 스크립트는 상위 9%까지 도달할 수 있을 만큼 충분한 0.808의 공개 LB 점수를 제공한다. 그러므로 나는 그 대본을 개선하고 추가할 여지가 많다고 확신한다.


In [8]:
#load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,
                             ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import KFold

Feature Exploration, Engineering and Cleaning

이제 우리는 대부분의 커널이 구조화된 방식과 같이 진행해 나갈 것이며, 그것은 먼저 손에 잡히는 데이터를 살펴보고, 가능한 feature enginering 기회를 식별하고, 어떤 범주형 특징을 수치적으로 인코딩하는 것이다

In [9]:

train = pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')

#store our passenger ID for easy access

PassengerId = test['PassengerId']


train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S



우리의 임무가 범주 형 변수에서 정보를 추출하는 것은 놀라운 일이 아닙니다.

Feature Engineering

여기에서 기능 엔지니어링 아이디어에 대한 Sina의 매우 포괄적이고 잘 고안된 노트북으로 크레딧을 연장해야하므로 그의 작업을 확인하십시오.

Titanic Best Working Classfier : by Sina
(https://www.kaggle.com/sinakhorami/titanic-best-working-classifier)




survival - Survival (0 = No; 1 = Yes)
class - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
name - Name
sex - Sex
age - Age
sibsp - Number of Siblings/Spouses Aboard
parch - Number of Parents/Children Aboard
ticket - Ticket Number
fare - Passenger Fare
cabin - Cabin (객실 번호)
embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
boat - Lifeboat (if survived)
body - Body number (if did not survive and body was recovered)

In [12]:
full_data = [train, test]

# some features of my own that I have added in
# gives the length of the name
train['Name_length'] = train['Name'].apply(len)
test['Name_length'] = test['Name'].apply(len)

#Feature that tells whether a passenger had 0 if type a cabin on the Tianic
# cabin = 선실

train['Has_Cabin'] = train["Cabin"].apply(lambda x:0 if type(x)== float else 1)
test['Has_Cabin'] = test["Cabin"].apply(lambda x:0 if type(x)==float else 1)

In [ ]:
#feature engineering steps taken from Sina

#Create new feature FamilySize as a combination of SibSp and Parch
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
    
# Remove all Nulls  in the Embarked column

for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')  #->?? 
    
#Remove all Nulls in the Fare column and create a new feature CategoricalFare
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())

train['CategoricalFare'] = pd.qcut(train['Fare'], 4)

# Create a new feature Categorical Age
for dataset in full_data:
    age_avg = dataset['Age'].mean()
    age_std = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    age_null_random_list = np.random.randint(age_avg - age_std, age_agv + age_std, size= age_null_count)
    
    # ????이게 뭐여...
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
    
train['CategoricalAge'] = pd.cut(train['Age'], 5)

dd
